In [1]:
import pandas as pd
import numpy as np

In [2]:
pop = pd.read_csv("../data/raw/nl3.csv", sep=',', index_col=[0])
dash = pd.read_csv('../data/processed/df_coo_pararius.csv', index_col=[0])
postcode = pd.read_csv("../data/raw/zipcode.csv", sep=',', index_col=[0])

In [3]:
searchfor = dash['City'].unique()
pop['Alternate Names'] = pop['Alternate Names'].fillna('None')
for i in searchfor:
    pop.loc[pop['Alternate Names'].str.contains(i), 'city'] = i
pop['city'] = pop['city'].fillna(pop['city'])
ast = list(set(searchfor) - set(pop['city']))
test = dash[dash['City'].isin(ast)]
postcode = postcode.rename(columns={'Plaats':'City'})

<ipython-input-3-7b6a62f44eb0>:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  pop.loc[pop['Alternate Names'].str.contains(i), 'city'] = i


In [4]:
display(test[['City', 'Price']])
display(postcode[['City', 'Provincie','longitude', 'latitude']])

,City,Price
16,Wehl,3500.0
26,Munstergeleen,2600.0
91,Terheijden,1175.0
130,Breukeleveen,1450.0
209,Zwanenburg,1245.0
...,...,...
4390,0,1635.0
4391,0,1605.0
4699,Giessen,1175.0
4715,0,945.0


,City,Provincie,longitude,latitude
0,Amersfoort,Utrecht,5.379921,52.198751
1,Nijmegen,Gelderland,5.866089,51.837674
2,IJmuiden,Noord-Holland,4.618372,52.459108
3,Utrecht,Utrecht,5.127076,52.088269
4,Hengelo,Overijssel,6.796709,52.260323
...,...,...,...,...
7719,Schiedam,Zuid-Holland,4.397530,51.911732
7720,Rotterdam,Zuid-Holland,4.487535,51.916806
7721,Utrecht,Utrecht,5.137477,52.088324
7722,Roosendaal,Noord-Brabant,4.461922,51.539267


In [5]:
aaa = pd.merge(test[['City', 'Price']], postcode[['City', 'Provincie','longitude', 'latitude']], left_on='City', right_on='City', how='left')

In [6]:
ccc = aaa.groupby(['City', 'Provincie'])[['longitude', 'latitude']].mean().reset_index()

In [7]:


def haversine(lat1, lon1, lat2, lon2):
    earth_radius = 6371
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
    a = np.sin((lat2-lat1)/2.0)**2 + np.cos(lat1) * \
        np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2
    return earth_radius * 2 * np.arcsin(np.sqrt(a))


In [8]:
ccc['latitude1'] = 51.907183
ccc['longitude1'] = 4.472815
ccc['distance'] = haversine(ccc['latitude'], ccc['longitude'], ccc['latitude1'], ccc['longitude1'])
ccc = ccc.rename(columns={'City': 'city', 'Provincie': 'Province'})

In [9]:
finaleira = pd.concat([pop, ccc], 0).reset_index(drop=True)


<ipython-input-9-d0442445b366>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  finaleira = pd.concat([pop, ccc], 0).reset_index(drop=True)


In [10]:
def haversine(lat1, lon1, lat2, lon2):
    earth_radius = 6371
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
    a = np.sin((lat2-lat1)/2.0)**2 + np.cos(lat1) * \
        np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2
    return earth_radius * 2 * np.arcsin(np.sqrt(a))


In [11]:
finaleira['latitude1'] = 51.907183
finaleira['longitude1'] = 4.472815

In [12]:
finaleira['distance'] = haversine(finaleira['latitude'], finaleira['longitude'], finaleira['latitude1'], finaleira['longitude1'])

In [13]:
finaleira = finaleira.sort_values('distance')

In [14]:
finaleira['cost'] = finaleira['cost'].interpolate().fillna(finaleira['cost'].mean())


In [15]:
dash.columns

Index(['deal', 'Garden', 'Add', 'Price', 'link', 'agency', 'Area', 'interior',
       'latitude', 'longitude', 'Rooms', 'City', 'Provincie', 'url', 'date',
       'status', 'irl', 'image', 'address', 'street', 'Offered', 'Available'],
      dtype='object')

In [16]:
#finaleira.to_csv("../data/raw/nl3.csv")
average_rent = dash.groupby(['City', 'Provincie'])[['Price']].mean().reset_index().rename(columns={'Price': 'avg_rent'})
fim = pd.merge(finaleira, average_rent, left_on=['city', 'Province'], right_on=['City', 'Provincie'], how='left')
fim['avg_costs'] = fim['avg_rent'] + fim['cost']
fim['latitude'] = round(fim['latitude'].astype(float), 5)
fim['longitude'] = round(fim['longitude'].astype(float), 5)
fim['distance'] = round(fim['distance'], 2)
fim['cost'] = round(fim['cost'], 2)
fim['avg_rent'] = round(fim['avg_rent'], 2)
fim['avg_costs'] = round(fim['avg_costs'], 2)

In [17]:

final = fim[[
    'Alternate Names',
    'city',
    'Province',
    'Population',
    'distance',
    'cost',
    'avg_rent',
    'avg_costs',
    'latitude',
    'longitude',
]].fillna('None')
searchfor = ["'s-Hertogenbosch",
             'Alkmaar',
             'Almere',
             'Amersfoort',
             'Amstelveen',
             'Amsterdam',
             'Apeldoorn',
             'Arnhem',
             'Breda',
             'Bussum',
             'Delft',
             'Dordrecht',
             'Eindhoven',
             'Gouda',
             'Haarlem',
             'Hilversum',
             'Hoofddorp',
             'Huizen',
             'Leiden',
             'Oegstgeest',
             'Rijswijk',
             'Rotterdam',
             'Schiedam',
             'Den Haag',
             'Utrecht',
             'Voorburg',
             'Zaandam',
             'Zeist',
             'Zoetermeer']
final[final['city'].isin(searchfor)].reset_index(drop=True).to_csv('../app/costs.csv')
dash = pd.read_csv('../data/processed/df_coo_pararius.csv', index_col=[0])
dash.to_csv('../app/df_coo_pararius.csv')

In [18]:
dash.columns

Index(['deal', 'Garden', 'Add', 'Price', 'link', 'agency', 'Area', 'interior',
       'latitude', 'longitude', 'Rooms', 'City', 'Provincie', 'url', 'date',
       'status', 'irl', 'image', 'address', 'street', 'Offered', 'Available'],
      dtype='object')